# Data Preparation
* **Project:** Mitochondrial 2158T>C variant in PD
* **Version:** Python/3.9
* **Status:** COMPLETE
* **Last Updated:** 14-MARCH-2024

### Notebook Overview
- Perform GenomeAnalysisToolkit to genotype chrM from WGS of 922 PD cases and 229 controls in GP2 monogenic cohort

In [ ]:
# Download reference genome and index
!gsutil -o 'GSUtil:parallel_thread_count=30' -u {BILLING_PROJECT_ID} -m cp -r gs://gcp-public-data--broad-references/hg38/v0/Homo_sapiens_assembly38.fasta .
!gsutil -o 'GSUtil:parallel_thread_count=10' -u {BILLING_PROJECT_ID} -m cp -r gs://gcp-public-data--broad-references/hg38/v0/Homo_sapiens_assembly38.fasta.fai .     

In [ ]:
#Download CRAM and index files available in GP2 monogenic cohort releaase 6, perform variant calling and delete CRAM files
!for i in `cat list_of_cram_prefix_1`  ; do \
    gsutil -o 'GSUtil:parallel_thread_count=30' -u {BILLING_PROJECT_ID} -m cp -r gs://{AMP-PD-GENOMICS}/samples/wgs/gatk/cram/PD-"$i"/PD-"$i".cram* . ; \
    gatk --java-options "-Xmx20g" Mutect2 -R Homo_sapiens_assembly38.fasta -L chrM  --mitochondria-mode -I PD-"$i".cram -O M_PD-"$i".cram.vcf.gz ; \
    rm PD-"$i".cram ; \
done 

In [ ]:
#Copy the results to the bucket
import os
BILLING_PROJECT_ID = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE = os.environ['WORKSPACE_NAME']
bucket = os.environ['WORKSPACE_BUCKET']
!gsutil cp ./*cram.vcf.gz $bucket